In [4]:
import os
import polars as pl
import pandas as pd
import sgml, dproc

In [2]:
if not os.path.isfile('data/train.csv'):
    !kaggle competitions download -c playground-series-s4e7
    !mkdir data
    !unzip playground-series-s4e7.zip -d data
    !rm playground-series-s4e7.zip
    !kaggle datasets download -d annantkumarsingh/health-insurance-cross-sell-prediction-data
    !unzip health-insurance-cross-sell-prediction-data.zip
    !mv train.csv data/train_org.csv
    !mv test.csv data/test_org.csv
    !rm health-insurance-cross-sell-prediction-data.zip

In [5]:
dfl_train = pl.read_csv('data/train.csv')

id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
i64,str,i64,i64,f64,i64,str,str,f64,f64,i64,i64
0,"""Male""",21,1,35.0,0,"""1-2 Year""","""Yes""",65101.0,124.0,187,0
1,"""Male""",43,1,28.0,0,"""> 2 Years""","""Yes""",58911.0,26.0,288,1
2,"""Female""",25,1,14.0,1,"""< 1 Year""","""No""",38043.0,152.0,254,0
3,"""Female""",35,1,1.0,0,"""1-2 Year""","""Yes""",2630.0,156.0,76,0
4,"""Female""",36,1,15.0,1,"""1-2 Year""","""No""",31951.0,152.0,294,0
…,…,…,…,…,…,…,…,…,…,…,…
11504793,"""Male""",48,1,6.0,0,"""1-2 Year""","""Yes""",27412.0,26.0,218,0
11504794,"""Female""",26,1,36.0,0,"""< 1 Year""","""Yes""",29509.0,152.0,115,1
11504795,"""Female""",29,1,32.0,1,"""< 1 Year""","""No""",2630.0,152.0,189,0


In [15]:
df_type = dproc.get_type_df(dfl_train)
df_type

,min,max,na,count,n_unique,dtype,f32,i32,i16,i8
feature,,,,,,,,,,
id,0.0,11504797.0,0.0,11504798.0,11504798.0,Int64,True,True,False,False
Gender,NaN,NaN,0.0,11504798.0,2.0,String,NaN,NaN,NaN,NaN
Age,20.0,85.0,0.0,11504798.0,66.0,Int64,True,True,True,True
Driving_License,0.0,1.0,0.0,11504798.0,2.0,Int64,True,True,True,True
Region_Code,0.0,52.0,0.0,11504798.0,54.0,Float64,True,True,True,True
Previously_Insured,0.0,1.0,0.0,11504798.0,2.0,Int64,True,True,True,True
Vehicle_Age,NaN,NaN,0.0,11504798.0,3.0,String,NaN,NaN,NaN,NaN
Vehicle_Damage,NaN,NaN,0.0,11504798.0,2.0,String,NaN,NaN,NaN,NaN
Annual_Premium,2630.0,540165.0,0.0,11504798.0,51728.0,Float64,True,True,False,False


Region_Code to category, Policy_Sales_Channel to category

In [61]:
pl_type = dproc.get_type_pl(df_type, {'Region_Code': pl.Categorical, 'Policy_Sales_Channel': pl.Categorical, 'Annual_Premium': pl.Categorical})

In [62]:
dfl_train = pl.read_csv('data/train.csv', dtypes = pl_type)

In [63]:
df_train = dfl_train.to_pandas().set_index('id')
df_train.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,,,,,,,,,,,
0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [64]:
dfl_test = pl.read_csv('data/train.csv', dtypes = pl_type)

In [76]:
df_test = dfl_test.to_pandas().set_index('id').sample(frac=0.3)
df_test.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,,,,,,,,,,,
3930590,Male,41,1,28.0,0,1-2 Year,Yes,35307.0,125.0,241,0
5875142,Female,22,1,4.0,0,< 1 Year,Yes,29241.0,124.0,36,1
866700,Male,50,1,28.0,0,> 2 Years,Yes,45915.0,26.0,56,0
10156976,Male,39,1,7.0,0,1-2 Year,Yes,45311.0,124.0,145,0
3053600,Male,20,1,41.0,1,< 1 Year,Yes,2630.0,160.0,262,0


In [34]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11504798 entries, 0 to 11504797
Data columns (total 11 columns):
 #   Column                Dtype   
---  ------                -----   
 0   Gender                category
 1   Age                   int8    
 2   Driving_License       int8    
 3   Region_Code           category
 4   Previously_Insured    int8    
 5   Vehicle_Age           category
 6   Vehicle_Damage        category
 7   Annual_Premium        float32 
 8   Policy_Sales_Channel  category
 9   Vintage               int16   
 10  Response              int8    
dtypes: category(5), float32(1), int16(1), int8(4)
memory usage: 219.4 MB


In [35]:
df_train['Gender'].cat.codes

id
0           0
1           0
2           1
3           1
4           1
           ..
11504793    0
11504794    1
11504795    1
11504796    1
11504797    0
Length: 11504798, dtype: int8

In [66]:
df_train['Annual_Premium'].dtype

CategoricalDtype(categories=['65101.0', '58911.0', '38043.0', '2630.0', '31951.0',
                  '28150.0', '27128.0', '40659.0', '31639.0', '27996.0',
                  ...
                  '56775.0', '70697.0', '60985.0', '64057.0', '64871.0',
                  '80920.0', '59411.0', '77189.0', '113339.0', '2820.0'],
, ordered=False, categories_dtype=object)

In [67]:
df_test['Annual_Premium'].dtype

CategoricalDtype(categories=['65101.0', '58911.0', '38043.0', '2630.0', '31951.0',
                  '28150.0', '27128.0', '40659.0', '31639.0', '27996.0',
                  ...
                  '14262.0', '83196.0', '78005.0', '60552.0', '68748.0',
                  '19611.0', '59601.0', '3620.0', '18776.0', '4061.0'],
, ordered=False, categories_dtype=object)

In [77]:
set(df_train['Annual_Premium'].unique()) - set(df_test['Annual_Premium'].unique())

{'19317.0',
 '250072.0',
 '56661.0',
 '79515.0',
 '74950.0',
 '20432.0',
 '5744.0',
 '71366.0',
 '57034.0',
 '66798.0',
 '111671.0',
 '76392.0',
 '66623.0',
 '76585.0',
 '78805.0',
 '63753.0',
 '13455.0',
 '6217.0',
 '74838.0',
 '19833.0',
 '18552.0',
 '65976.0',
 '82878.0',
 '67730.0',
 '57089.0',
 '69760.0',
 '62899.0',
 '66394.0',
 '19348.0',
 '55772.0',
 '53073.0',
 '19439.0',
 '19655.0',
 '64106.0',
 '56144.0',
 '64217.0',
 '119148.0',
 '55743.0',
 '19034.0',
 '49075.0',
 '72310.0',
 '95407.0',
 '64543.0',
 '74463.0',
 '57774.0',
 '62670.0',
 '21629.0',
 '69704.0',
 '66541.0',
 '67435.0',
 '71679.0',
 '18503.0',
 '15921.0',
 '21561.0',
 '70960.0',
 '139130.0',
 '51704.0',
 '61068.0',
 '18216.0',
 '259841.0',
 '57076.0',
 '68721.0',
 '16266.0',
 '81027.0',
 '99324.0',
 '62730.0',
 '50025.0',
 '3031.0',
 '57919.0',
 '53617.0',
 '73721.0',
 '63160.0',
 '147075.0',
 '439017.0',
 '56392.0',
 '62052.0',
 '70250.0',
 '62163.0',
 '72863.0',
 '63716.0',
 '13853.0',
 '55697.0',
 '64891.0',


In [78]:
df_test.loc[df_test['Annual_Premium'] == df_train.iloc[0]['Annual_Premium']]

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,,,,,,,,,,,
4415567,Male,29,1,35.0,0,< 1 Year,Yes,65101.0,152.0,129,1
1772830,Male,78,1,28.0,0,1-2 Year,Yes,65101.0,26.0,71,0
1221372,Female,21,1,28.0,1,< 1 Year,No,65101.0,160.0,254,0
5105509,Male,22,1,35.0,1,< 1 Year,No,65101.0,160.0,129,0


In [50]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output =False)
ohe.fit(df_train[['Gender']])

OneHotEncoder(sparse_output=False)

In [51]:
ohe.transform(df_test[['Gender']])

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [70]:
import lightgbm as lgb

df_t = pd.concat([
    df_train[['Annual_Premium']].assign(is_train = True),
    df_test[['Annual_Premium']].assign(is_train = False)
])

In [79]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
ss = ShuffleSplit(1, train_size=0.8)
cross_val_score(lgb.LGBMClassifier(verbose=-1), df_t[['Annual_Premium']], df_t['is_train'], 
                params={'categorical_feature': ['Annual_Premium']}, scoring='roc_auc', cv= ss)

array([0.47624251])

In [55]:
clf_lgb = lgb.LGBMClassifier(verbose=-1)
clf_lgb.fit(df_train[['Gender']], df_train

Init signature:
ShuffleSplit(
    n_splits=10,
    *,
    test_size=None,
    train_size=None,
    random_state=None,
)
Docstring:     
Random permutation cross-validator.

Yields indices to split data into training and test sets.

Note: contrary to other cross-validation strategies, random splits
do not guarantee that all folds will be different, although this is
still very likely for sizeable datasets.

Read more in the :ref:`User Guide <ShuffleSplit>`.

For visualisation of cross-validation behaviour and
comparison between common scikit-learn split methods
refer to :ref:`sphx_glr_auto_examples_model_selection_plot_cv_indices.py`

Parameters
----------
n_splits : int, default=10
    Number of re-shuffling & splitting iterations.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    